# Data analysis on "Immobiliare.it" data.
After some feature engineering and exploratory analysis, different models will be tried to identify the logic that will best fit the task.

### Feature exploration
The data will be loaded and the features explored.

In [1]:
# Importing the necessary libraries
import pandas as pd
import numpy as np

In [2]:
houses = pd.read_csv("DATA/casemilano.csv", encoding='unicode_escape')

In [3]:
#Missing data will be removed
#This leads to lose about 35% of observations, but it must be done, as the variables with missing values are important ones, and cannot be excluded.
houses.dropna(inplace=True)

In [4]:
houses.shape

(7297, 18)

In [5]:
# The features will be cleaned.

#Features eliminated
houses = houses.drop('w', axis=1) #It's a constant variable
houses = houses.drop("s", axis=1) #It's unclear the meaning of the variable

#Ascensore recoded.
houses.loc[houses["ascensore"] == "no", "ascensore"] = 0
houses.loc[houses["ascensore"] != 0, "ascensore"] = 1
houses["ascensore"].astype(np.int64)

#Parking: more than 2 in shared parking space are merged.

#Features made character. The dtype remain "object"
houses["numero.bagni"] = houses["numero.bagni"].astype(str)
houses["numero.stanze"] = houses["numero.stanze"].astype(str)

#Features made numeric
houses["numero.totale.piani.edificio"][houses["numero.totale.piani.edificio"] == "1 piano"] = 1
houses["numero.totale.piani.edificio"] = houses["numero.totale.piani.edificio"].astype(np.int8)

In [15]:
houses.head()

,prezzo,metri.quadrati,numero.bagni,ascensore,numero.stanze,altre.caratteristiche,numero.totale.piani.edificio,parcheggio,disponibile,spese.condominiali,anno.di. Costruzione,condizioni,zona,piano,riscaldamento.centralizzato,classe.di.efficienza.energetica
1,500000.0,57.0,1,1,2,fibra ottica | videocitofono | sistema d'allar...,7,1 in garage/box,disponibile,nessun costo condominiale,2010.0,eccellente / restaurato,porta romana - medaglie d'oro,1,centralizzato,e
2,395000.0,92.0,1,1,3,cancello elettrico | fibra ottica | videocitof...,8,no,disponibile,200,1960.0,eccellente / restaurato,gallaratese,1,centralizzato,e
4,199000.0,38.0,1,0,1,fibra ottica | porta di sicurezza | esposizion...,4,no,disponibile,80,1930.0,eccellente / restaurato,navigli - darsena,1,indipendente,g
5,299000.0,45.0,1,0,1,cancello elettrico | fibra ottica | porta di s...,5,no,disponibile,100,1911.0,buone condizioni /vivibile,porta romana - medaglie d'oro,1,centralizzato,g
9,1290000.0,170.0,2,1,4,porta di sicurezza | terrazza | terrazza |port...,6,no,disponibile,500,1970.0,buone condizioni /vivibile,de angeli,6,centralizzato,g


In [16]:
houses["parcheggio"].value_counts()

parcheggio
no                           5617
1 in garage/box              1066
1 in parcheggio condiviso     388
2 in garage/box               192
2 in parcheggio condiviso      23
4 in parcheggio condiviso       5
3 in parcheggio condiviso       3
5 in parcheggio condiviso       1
9 in parcheggio condiviso       1
6 in parcheggio condiviso       1
Name: count, dtype: int64

In [14]:
houses.dtypes

prezzo                             float64
metri.quadrati                     float64
numero.bagni                        object
ascensore                           object
numero.stanze                       object
altre.caratteristiche               object
numero.totale.piani.edificio          int8
parcheggio                          object
disponibile                         object
spese.condominiali                  object
anno.di. Costruzione               float64
condizioni                          object
zona                                object
piano                               object
riscaldamento.centralizzato         object
classe.di.efficienza.energetica     object
dtype: object